In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

초기설정

In [ ]:
# 웹 드라이버 설정 (Chrome 사용 예제)
options = Options()
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 웹 페이지 열기
url = 'https://korean.visitkorea.or.kr/list/travelinfo.do?service=ms'
driver.get(url)
driver.maximize_window()

함수 선언부

In [ ]:
def button_click():
    # "인기순" 버튼 클릭
    popular_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "인기순")]')))
    driver.execute_script("arguments[0].click();", popular_button)

    # 서울 버튼 클릭
    seoul_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="1"]/button/span')))
    driver.execute_script("arguments[0].click();", seoul_button)

    # 서울 버튼 클릭 후, 해당 지역의 데이터가 로드될 때까지 대기
    wait = WebDriverWait(driver, 30)
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/div[2]/div[1]/ul/li[1]/div[2]/div/a')))


# 각 페이지 데이터 추출 함수
# 이동한 페이지의 데이터를 추출하기 위해, 이동한 페이지의 html 소스를 가져와 파싱
def extract_data(soup, attraction_list, location_list, tag_list):
    for i in range(10):
        attraction = soup.select_one(f'#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li:nth-child({i+1}) > div.area_txt.catchphrase > div > a')
        location = soup.select_one(f'#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li:nth-child({i+1}) > div.area_txt.catchphrase > p:nth-child(2)')
        tag = soup.select_one(f'#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li:nth-child({i+1}) > div.area_txt.catchphrase > p.tag')

        # 명소, 위치, 해쉬 태그 3개를 다 가진 것만 추출
        if attraction and location and tag:
            attraction_list.append(attraction.text)
            location_list.append(location.text)
            tag_list.append(tag.text)

# 각 페이지 스크롤 내리는 함수
def slow_scroll_by(offset, duration):
    steps = 30  # 스크롤을 나누는 단계 수
    step_duration = duration / steps  # 각 단계별 대기 시간
    step_size = offset / steps  # 각 단계별 스크롤 크기
    for _ in range(steps):
        driver.execute_script(f"window.scrollBy(0, {step_size})") # steps 동안 step_size 만큼 반복해서 스크롤
        time.sleep(step_duration) # 1회 스크롤 하고 step_duration 만큼 대기

# 다음 페이지로 넘어가는 함수
def next_page(page_number):
    # 페이지 바에서 모든 페이지 링크 요소를 찾음
    page_bar = driver.find_elements(By.CSS_SELECTOR, '.page_box a')

    # 찾은 페이지 링크 요소들 중에서 지정된 페이지 번호(page_number)와 일치하는 버튼을 클릭
    for button in page_bar:
        if button.get_attribute("id") == str(page_number):
            # element.click() 기본 클릭 메소드
            # 동적 콘텐츠로 로드하는 경우, 기본 클릭 메소드가 작동되지 않음
            # 자바스크립트를 사용하여 클릭 이벤트 실행
            driver.execute_script("arguments[0].click();", button)

            break
    # 페이지가 로드되기 전에 대기
    wait = WebDriverWait(driver, 20)
    # 페이지의 첫 번째 링크 요소가 새로운 페이지 로딩으로 인해 사라질 때까지 대기
    wait.until(EC.staleness_of(page_bar[0]))
    # 새 페이지가 로드된 후, 특정 요소가 존재하는지 확인하여 페이지 로딩 완료를 확인
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/div[2]/div[1]/ul/li[1]/div[2]/div/a')))
    # 페이지 로딩 후 잠시 대기
    time.sleep(1.5)

# 한 면에서 넘길 수 있는 페이지 수는 5개
# 그 이후에는 다음 버튼을 클릭해야함.
# 다음 버튼 클릭 함수
def click_next_button():
    # CSS 셀렉터.btn_next.ico를 사용하여 다음 버튼을 찾음
    # CSS 셀렉터는 <a> 태그의 class 속성이 btn_next와 ico를 가진 요소를 찾음
    next_button = driver.find_element(By.CSS_SELECTOR, '.btn_next.ico')
    # 자바스크립트를 사용하여 클릭 이벤트 실행
    driver.execute_script("arguments[0].click();", next_button)
    # 페이지 로드 대기
    wait = WebDriverWait(driver, 20)
    wait.until(EC.staleness_of(next_button))
    # 새로운 페이지 로드 대기:
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/div[2]/div[1]/ul/li[1]/div[2]/div/a')))
    time.sleep(1.5)

# 페이지 바 요소
# <div class="page_box">
# <a class="on" title="선택됨" href="javascript:" id="1">1</a>
# <a href="javascript:" id="2">2</a>
# <a href="javascript:" id="3">3</a>
# <a href="javascript:" id="4">4</a>
# <a href="javascript:" id="5">5</a>
# <a href="javascript:" class="btn_next ico" id="6">다음</a>
# <a class="btn_last ico" href="javascript:" id="3937">끝</a></div>


코드 작성부

In [ ]:
try:
    # 인기순, 서울 버튼 클릭 함수
    button_click()

    # 대이터 저장 리스트 초기화
    attraction_list = []
    location_list = []
    tag_list = []

    # 첫 번째 페이지의 HTML 소스를 가져와 BeautifulSoup으로 파싱
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')

    # 첫번째 페이지 추출
    extract_data(soup, attraction_list, location_list, tag_list)

    # 2페이지부터 페이지 이동 및 데이터 추출 반복
    for p in range(2, 15):  # 예시로 15페이지까지
        slow_scroll_by(1600, 3) # 위 스크롤 함수 이용
        next_page(p)    # 페이지 이동 버튼 클릭 함수

        # 이동한 페이지의 HTML 소스를 가져와 BeautifulSoup으로 파싱
        content = driver.page_source
        soup = BeautifulSoup(content, 'html.parser')

        extract_data(soup, attraction_list, location_list, tag_list)

        # 페이지 번호가 5의 배수일 때 "다음" 버튼 클릭
        if p % 5 == 0:
            click_next_button()

    # 데이터프레임 생성 및 출력
    data = {'attraction': attraction_list, 'location': location_list, 'tag': tag_list}
    df = pd.DataFrame(data)
    print(df)

finally:
    # 드라이버 종료
    driver.quit()

             attraction location  \
0             동대문역사문화공원    서울 중구   
1                 서울식물원   서울 강서구   
2        동대문디자인플라자(DDP)    서울 중구   
3                남산서울타워   서울 용산구   
4                   서울숲   서울 성동구   
..                  ...      ...   
131              중앙고등학교   서울 종로구   
132         북한산국립공원(서울)   서울 성북구   
133  여의도 둘레길(여의도 자전거도로)  서울 영등포구   
134        영등포 신길동 홍어거리  서울 영등포구   
135        동대문종합시장 한복상가   서울 종로구   

                                                   tag  
0    #2024_여행가는달과함께하는_발도장여행지#DDP#가족여행#걷기좋은길#관광지#데이트...  
1    #2024_여행가는달과함께하는_발도장여행지#가족과함께#관광지#교육여행#미세먼지_피하...  
2    #19_20한국관광100선#2024_여행가는달과함께하는_발도장여행지#21_22한국관...  
3    #19_20한국관광100선#2024_여행가는달과함께하는_발도장여행지#21_22한국관...  
4    #2024_여행가는달과함께하는_발도장여행지#23_24한국관광100선#가족여행#걷기길...  
..                                                 ...  
131  #고려중앙학원#관광지#기호학교#문화재#사적지#서울중앙고등학교#수도권#역사#역사공부#...  
132  #관광지#북한산#북한산국립공원#생태#생태관광#생태관광지#서울권#수도권#자연#자연#자...  
133  #레포츠#산책코스#샛강생태공원#수변산책로#여의도#여의도한강공원#자전거둘레길#자전거코...

In [ ]:
# 결과물 확인
df[:10]

,attraction,location,tag
0,동대문역사문화공원,서울 중구,#2024_여행가는달과함께하는_발도장여행지#DDP#가족여행#걷기좋은길#관광지#데이트...
1,서울식물원,서울 강서구,#2024_여행가는달과함께하는_발도장여행지#가족과함께#관광지#교육여행#미세먼지_피하...
2,동대문디자인플라자(DDP),서울 중구,#19_20한국관광100선#2024_여행가는달과함께하는_발도장여행지#21_22한국관...
3,남산서울타워,서울 용산구,#19_20한국관광100선#2024_여행가는달과함께하는_발도장여행지#21_22한국관...
4,서울숲,서울 성동구,#2024_여행가는달과함께하는_발도장여행지#23_24한국관광100선#가족여행#걷기길...
5,경복궁,서울 종로구,#15_16한국관광100선#17_18한국관광100선#19_20한국관광100선#202...
6,인사동,서울 종로구,#관광지#도심여행#수도권#시티투어#인사동#주말추천여행지#체험
7,익선동,서울 종로구,#21_22한국관광100선#23_24한국관광100선#가족과함께#관광지#남녀노소#뉴트...
8,명동,서울 중구,#15_16한국관광100선#17_18한국관광100선#19_20한국관광100선#관광지...
9,롯데월드 어드벤처,서울 송파구,#17_18한국관광100선#19_20한국관광100선#2024_여행가는달과함께하는_발...
